In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.python.framework import ops

from mini_batch import mini_batch


In [136]:
from utils import *

In [137]:
X_train, y_train, X_test, y_test = load_dataset(dataset_name="opportunity")

 ..from file /Users/chentailin/Desktop/workspace/paper_multimodality/Opportunity/data/activity-dataset (113_wearable+40_ambient).data
 ..reading instances: x_train (557963, 153), x_test (118750, 153)
 ..reading instances: y_train (557963,), y_test (118750,)


# Frame-wise 
Sliding Window

In [138]:
print("opperating sliding window!")
X_train, y_train = op_sliding_window(X_train,y_train,24,
                                     12)
X_test, y_test = op_sliding_window(X_test, y_test,24,12)

opperating sliding window!
Sliding Window Successfully 
Input shape is (557963, 153) and (46495, 1, 24, 153),
Output shape is (557963,) and (46495, 1).
Sliding Window Successfully 
Input shape is (118750, 153) and (9894, 1, 24, 153),
Output shape is (118750,) and (9894, 1).


In [139]:
X_train = np.expand_dims(X_train,-1)
X_test = np.expand_dims(X_test,-1)

In [140]:
y_train = np.expand_dims(y_train,-1)
y_test = np.expand_dims(y_test,-1)

In [141]:
y_train = one_hot_encoder(y=y_train,num_classes=18)

y_test = one_hot_encoder(y_test,num_classes=18)


In [142]:
y_test.shape

(9894, 18)

# Sample-wise

In [143]:
num_classes = 18
num_hidden_lstm = 32
X_train.shape

(46495, 24, 153, 1)

In [144]:
print('X_train.shape' + str(X_train.shape))
print('Y_train.shape' + str(y_train.shape))

X_train.shape(46495, 24, 153, 1)
Y_train.shape(46495, 18)


In [145]:
def create_placeholders(dim, win_len, n_C0, num_classes):
    """
    Arguments:
    dim -- scalar, dimension of the input (52)
    win_len -- scalar, window_size of the input (100)
    n_C0 -- scalar, number of channels of the input (1)
    num_classes -- scalar, number of classes (12)
    
    Returns:
    X -- placeholder for the data input, of shape [None, win_len, dim, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, num_classes] and dtype "float"
    """
    X = tf.placeholder(tf.float32, shape=(None, dim, win_len, n_C0))
    Y = tf.placeholder(tf.float32, shape=(None, num_classes))
    
    return X, Y

In [146]:
def initialize_parameters():
    """
    Initializes weight parameters to build a cnn with tensorflow. The shapes are:
                        W1 : [1, 5, 1,16]
                        W2 : [1, 5, 16, 16]
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    W1 = tf.get_variable("W1", [1, 5, 1,16], initializer = tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable("W2", [1, 5, 16, 16], initializer = tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [num_hidden_lstm, num_classes], initializer = tf.contrib.layers.xavier_initializer())
    by = tf.get_variable("by", [num_classes], initializer = tf.zeros_initializer())


    parameters = {"W1": W1,
                  "W2": W2,
                  "Wy": Wy,
                  "by": by}
    
    return parameters

In [157]:
def forward_propagation(X, parameters, keep_prob):
    """
    Arguments:
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    ZL -- the output of the last LINEAR unit
    """
    W1 = parameters['W1']
    W2 = parameters['W2']
    Wy = parameters['Wy']
    by = parameters['by']
    X = tf.cast(X, tf.float32)
    print('forward_propagation1-----' + str(X))
    Z1 = tf.nn.conv2d(X,W1, strides = [1,1,1,1], padding = 'SAME')
    A1 = tf.nn.relu(Z1)
    print('forward_propagation2-----' + str(A1))
    
    P1 = tf.nn.max_pool(A1, ksize = [1,1,2,1], strides = [1,1,2,1], padding = 'SAME')
    print('forward_propagation3-----' + str(P1))
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME')
    print('forward_propagation4-----' + str(Z2))
    A2 = tf.nn.relu(Z2)
    print('forward_propagation5-----' + str(A2))
    P2 = tf.nn.max_pool(A2, ksize = [1,1,2,1], strides = [1,1,2,1], padding = 'SAME')
    print('forward_propagation6-----' + str(P2))
    
    print()
    
    Reshape_1 = tf.transpose(P2, perm=[0,2,1,3])
    Reshape_2 = tf.reshape(Reshape_1,[-1,24,39*16])
    
    
    inputs = tf.unstack(Reshape_2, 24, 1)
    
    
    # def LSTM():
    
    # lstm cell 
    
    lstm_layer = rnn.BasicLSTMCell(num_hidden_lstm,forget_bias=1)
    
    outputs, _ = rnn.static_rnn(lstm_layer,inputs,dtype="float32")
    
    output = outputs[-1]
    
    
    ZL = tf.matmul(output, parameters['Wy']) + parameters['by']
    
    
    return ZL

In [158]:
def compute_cost(ZL, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = ZL, labels = Y))
    
    return cost

In [159]:
def compute_cost(ZL, Y):
    
    
#     l2 = lambda_loss_amount * sum(
#     tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
#     )
    
    cost =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=ZL,labels=Y)) 
    
    return cost

In [160]:
def model(X_train, y_train, X_valid, y_valid, learning_rate = 0.001,
          num_epochs = 15, minibatch_size = 128, print_cost = True):
    
    ops.reset_default_graph() 
    
    (m, win_len,dim,  n_C0) = X_train.shape      
    n_y = y_train.shape[1]
        
    #create placeholders for X, Y and keep_prob     
    X, Y = create_placeholders( win_len,dim, n_C0, num_classes)
    keep_prob = tf.placeholder(tf.float32)
    
    #initialize parameters
    parameters = initialize_parameters()
    
    #get the prediction of Y labels
    prediction = forward_propagation(X, parameters ,keep_prob)
    
    #cost     
    cost = compute_cost(prediction,Y)
    
    #use Adam optimizer     
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    #keep track of accuracies and losses of test and train data set     
    test_losses = []
    test_accuracies = []
    train_losses = []
    train_accuracies = []
    
    #
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    #
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        step = 1
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.      

            minibatches = mini_batch(X_train,y_train,minibatch_size)
            
            for minibatch in minibatches:
                
                (minibatch_x, minibatch_y) = minibatch
            
            
                _, loss, acc = sess.run(
                [optimizer, cost, accuracy],feed_dict={X: minibatch_x, 
                                                       Y: minibatch_y, 
                                                       keep_prob: 0.5})
                train_losses.append(loss)
                train_accuracies.append(acc)
            
            
            # Evaluate network only at some steps for faster training: 
            if (epoch % 1 == 0):
        
                # To not spam console, show training accuracy/loss in this "if"
                print("epoch iter #" + str(epoch) + ":Batch Loss = " + "{:.6f}".format(loss) + ", Accuracy = {}".format(acc))
            
                # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
                loss, acc = sess.run(
                    [cost, accuracy], 
                    feed_dict={
                        X: X_valid,
                        Y: y_valid,
                        keep_prob: 1})
                
                test_losses.append(loss)
                test_accuracies.append(acc)
                print("PERFORMANCE ON TEST SET: " + \
                      "Batch Loss = {}".format(loss) + \
                      ", Accuracy = {}".format(acc))


        print("Optimization Finished!")
        
        # plot the cost
        apredictions, accuracy, final_loss = sess.run(
        [prediction, accuracy, cost],
        feed_dict={
            X: X_valid,
            Y: y_valid,
            keep_prob: 1})

        test_losses.append(final_loss)
        test_accuracies.append(accuracy)

        print("FINAL RESULT: " + \
            "Batch Loss = {}".format(final_loss) + \
            ", Accuracy = {}".format(accuracy))
        
        
        predictions = apredictions.argmax(1)

        print("Testing Accuracy: {}%".format(100*accuracy))




        return parameters,test_losses,test_accuracies,train_losses,train_accuracies

In [161]:
_,test_losses,test_accuracies,train_losses,train_accuracies = model(X_train, y_train, X_test, y_test)

forward_propagation1-----Tensor("Placeholder:0", shape=(?, 24, 153, 1), dtype=float32)
forward_propagation2-----Tensor("Relu:0", shape=(?, 24, 153, 16), dtype=float32)
forward_propagation3-----Tensor("MaxPool:0", shape=(?, 24, 77, 16), dtype=float32)
forward_propagation4-----Tensor("Conv2D_1:0", shape=(?, 24, 77, 16), dtype=float32)
forward_propagation5-----Tensor("Relu_1:0", shape=(?, 24, 77, 16), dtype=float32)
forward_propagation6-----Tensor("MaxPool_1:0", shape=(?, 24, 39, 16), dtype=float32)

epoch iter #0:Batch Loss = 0.192469, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 0.8973413109779358, Accuracy = 0.8325247764587402
epoch iter #1:Batch Loss = 0.220822, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 0.9005732536315918, Accuracy = 0.8325247764587402


KeyboardInterrupt: 

In [ ]:
X_train.shape